In [ ]:
%cd /data/marciano/experiments/multi-organ-qc/moqc

In [ ]:
import webbrowser
from IPython.display import HTML
# Specify the path to your HTML file
html_file_path = '/data/marciano/experiments/multi-organ-qc/logs/dcgan.html'

# Display the HTML file in the notebook
HTML(filename=html_file_path)

In [ ]:
from models.GAN.dcgan import DCGAN
from utils.preprocess import transform_aug
from utils.dataset import NiftiDataset
import torch
import torch.nn as nn
import yaml
import numpy   as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open('models/config.yml', 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)
parameters = config["parameters"]['dcgan']
parameters['functions'] = {
    "BCELoss": nn.BCELoss(),
}
model = DCGAN(**parameters).to(device)
model.load_checkpoint(data_path='/data/marciano/experiments/multi-organ-qc/data/prostate/', eval=True)
transform, _ = transform_aug(None, 'dcgan')
dataset = NiftiDataset('/data/marciano/experiments/multi-organ-qc/data/prostate/nnunet/structured', transform=transform)
test_dataloader = torch.utils.data.DataLoader(dataset, batch_size=200, shuffle=False, num_workers=0)

model.eval()  # Set the model to evaluation mode
total_loss = 0.0
correct = 0
total = 0
outputs = []
val_data = torch.cat([inputs for inputs in test_dataloader], dim=0).float().to(device)
emb = model.netD.embed(torch.tensor(val_data).to(device))
outputs = model.netG(torch.tensor(emb).to(device))
        

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(outputs[30].cpu().detach().numpy().squeeze(), cmap='gray')
plt.show()
plt.imshow(val_data[30].cpu().numpy().squeeze(), cmap='gray')